## Un ejemplo de la bifurcación de Hopf ##

Sean los parámetros $\mu\in\mathbb{R}$ y $\beta>0$ y el sistema
\begin{gather*}
    \left\{\begin{array}{l}
        \dot{x} = -\mu x + y + \dfrac{x}{\beta+x^2+y^2}\,,\\
        \dot{y} = -x -\mu y + \dfrac{y}{\beta+x^2+y^2}\,,
    \end{array}\right.
\end{gather*}
el cual toma la forma en coordenadas polares conduce a que el ángulo polar está dado por $\theta(t)=-t+\theta_0$, con $\theta_0\in\mathbb{R}$ y el radio satisface el sistema escalar
\begin{gather*}
        \dot{r} = r\left(-\mu+\dfrac{1}{\beta+r^2} \right)\,.
\end{gather*}

In [1]:
# Librerías necesarias: 
from numpy import *  
from scipy import *  
import scipy.linalg as la
from scipy.integrate import ode
import matplotlib.pyplot as plt         
from pylab import * 
from numpy import linalg as LA
#
rcParams['xtick.direction']  = 'out'
rcParams['ytick.direction']  = 'out'
rcParams['mathtext.fontset'] = 'cm'
rcParams['mathtext.rm']      = 'serif'
rcParams['text.usetex']      = True
rcParams['axes.labelsize']   = 26
rcParams['axes.titlesize']   = 24
rcParams['xtick.labelsize']  = 24
rcParams['ytick.labelsize']  = 24
rcParams['legend.fontsize']  = 24
#
rr = 0
#
if rr == 0:
    %matplotlib qt
else:
    %matplotlib inline
#

In [11]:
# Define time array and parameter value
#
mu   = 0.1#|0.1|,2.0,1e-06
beta = 1.0
#
t0 = 0
ht = 5e-02
#
# Define vector field
def fieldXY(t,u):
    x  = u[0]
    y  = u[1]
    F  = -mu*x + y + x/(beta+x**2+y**2)
    G  = -x - mu*y + y/(beta+x**2+y**2)
    FG = [F,G]
    return FG
#
# Define parameter values and initial condition
eps = 0.000001
#
if (mu == 0):
    ui = [[0,0],[-1,0]]
else:
    ui = [[0,0],[-sqrt(abs(1/mu-beta)),2*sqrt(abs(1/mu-beta))]]
#    
fig0 = plt.figure(0,figsize=(11,9), constrained_layout=True)
ax0  = fig0.add_subplot(111)
#
ax0.axvline(x=0,ls='--',lw=1,color='k')
ax0.axhline(y=0,ls='--',lw=1,color='k')
ax0.plot([0],[0],'rD',ms=14) #Equilibrium point
#
colour = ['C2','C4']
#
n = len(ui)
for jj in range(n):
    if (mu > 0) and (mu < 1/beta):
        # Solve and plot for initial condition
        tf  = 500
        inf = int((tf-10)/ht)
        u0  = ui[jj] + eps*rand(2)
        #
        ax0.text(4.5,5,r'$\alpha(\gamma)=p_*$',color='r',fontsize=26)
        ax0.text(0.2,0.3,r'$p_*$',color='r',fontsize=26)
        ax0.text(-4.5,5,r'$\omega(\gamma)=\Gamma$',color='r',fontsize=26)
        ax0.text(-2.7,2.1,r'$\Gamma$',color='r',fontsize=26)
    elif (mu < 0):
        tf = 60
        u0 = ui[0] + eps*array([0.01,0.01])
        #
        ax0.text(50,75,r'$\alpha(\gamma)=p_*$',color='r',fontsize=26)
        ax0.text(6,5.5,r'$p_*$',color='r',fontsize=26)
    elif (mu > 1/beta):
        tf = 50
        u0 = array([0.5,0.5])
        #
        ax0.text(0.1,0.4,r'$\omega(\gamma)=p_*$',color='r',fontsize=26)
        ax0.text(0.01,0.02,r'$p_*$',color='r',fontsize=26)
    else:
        tf = 500
    #
    u = ode(fieldXY)
    #u.set_integrator('vode', method='bdf', order=15, nsteps=3000)
    u.set_integrator('lsoda',atol=1e-12, rtol=1e-10)
    u.set_initial_value(u0,t0)
    # set solution array
    ts = []
    us = []
    #
    while u.successful() and (u.t < tf):
        u.integrate(u.t + ht)
        ts.append(u.t)
        us.append(u.y)
    #
    time   = np.vstack(ts)
    Ustate = np.vstack(us)
    #
    xx = Ustate[:,0]
    yy = Ustate[:,1]
    #
    ax0.plot(xx, yy,lw=2,alpha=0.7,color=colour[jj]) # orbit 
    ax0.plot(xx[0],yy[0],'o',ms=8,color=colour[jj]) #Initial point
#
if (mu > 0) and (mu < 1/beta):
    ax0.plot(xx[inf:], yy[inf:],color='r',lw=4) # limit cycle
#
ax0.set_xlabel('$x$')
ax0.set_ylabel('$y$')
ax0.set_title(r'$\mu=%0.2f$'%mu)
show()
#close()

In [12]:
# Define explicitly Jacobian matrix
def JField(u,mu,beta):
    x   = u[0]
    y   = u[1]
    Fx  = -mu + (beta-x**2+y**2)/(beta+x**2+y**2)**2
    Fy  = 1 - 2*x*y/(beta+x**2+y**2)**2
    Gx  = -1 - 2*x*y/(beta+x**2+y**2)**2
    Gy  = -mu + (beta+x**2-y**2)/(beta+x**2+y**2)**2
    JFG = [[Fx,Fy],[Gx,Gy]]
    J   = JFG
    return J
#
muvector = linspace(-5*mu,25*mu,45)
#
fig1 = plt.figure(0,figsize=(11,9), constrained_layout=True)
ax1  = fig1.add_subplot(111)
#
ax1.axvline(x=0,ls='--',lw=1,color='k')
ax1.axhline(y=0,ls='--',lw=1,color='k')
#
for mnu in muvector:
    JacMat = JField([0,0],mnu,beta)
    lam, v = LA.eig(JacMat)
    Re     = real(lam)
    Im     = imag(lam)
    #
    if (Re[0] == 0):
        marker = 'Dr'
        ms     = 12
        ax1.set_title('$\mu=%1.1f$'%mnu,color='r')
    else:
        marker = 'ok'
        ms     = 10
    #    
    ax1.plot(Re,Im,marker,ms=ms)
#
if (mu > 0) and (mu < 1/beta):
    ax1.text(0.75,1.15,r'$\lambda(\mu)$',color='k',fontsize=26)
    ax1.text(0.75,-1.25,r'$\bar{\lambda}(\mu)$',color='k',fontsize=26)
    #
ax1.set_xlabel('$\Re{(\lambda)}$')
ax1.set_ylabel('$\Im{(\lambda)}$')
ax1.set_ylim(-2,2)
show()
#close()

In [9]:
# Plot limit cycle radius as function of mu
#
betavector = [0.25,0.5,0.75,1.0]
colour     = ['C6','C0','C4','r']
#
rr = linspace(0,3.5,100)
#
fig2 = plt.figure(0,figsize=(11,9), constrained_layout=True)
ax2  = fig2.add_subplot(111)
#
ax2.axvline(x=0,ls='--',lw=1,color='k')
ax2.axhline(y=0,ls='--',lw=1,color='k')
#
bbeta = len(betavector)
for jj in range(bbeta):
    mmu = 1.0/(betavector[jj]+rr**2)
    #
    ax2.plot(mmu,rr,lw=2,color=colour[jj])
    if (betavector[jj] == 1):
        marker = 'or'
    else:
        marker = 'ob'
    #
    ax2.plot(1/betavector[jj],0,marker,ms=10)
#
rrmu = sqrt(1.0/mu-beta)
#
# cycle limit radius for chosen mu.
#
ax2.plot(mu,rrmu,'Dr',ms=8,label=r'$r^*=%1.2f$'%rrmu) 
ax2.plot(mu,rrmu,'Dr',ms=0,label=r'$\mu^*=%1.2f$'%mu)
ax2.text(0.6,-0.17,r'$\beta=1.0$',fontsize=24,color=colour[3])
ax2.text(1.2,-0.17,r'$\beta=0.75$',fontsize=24,color=colour[2])
ax2.text(1.9,-0.17,r'$\beta=0.5$',fontsize=24,color=colour[1])
ax2.text(3.5,-0.17,r'$\beta=0.25$',fontsize=24,color=colour[0])
ax2.legend(loc='best',frameon=False)
ax2.set_ylim(-0.3,3.5)
ax2.set_xlim(-0.4,4.2)
#
ax2.set_xlabel('$\mu$')
ax2.set_ylabel('$r^*$')
show()
#close()

In [13]:
N = 300
bbe = linspace(0,2,N)
mmu = linspace(0.05,1/bbe[-1],N)
#
MU, BET = meshgrid(mmu,bbe)
#
rstar = sqrt(1/MU-BET)
#
fig3 = plt.figure(0,figsize=(11,9), constrained_layout=True)
ax3  = fig3.add_subplot(111)
#
cs0  = ax3.contourf(MU,BET,rstar,levels=25,cmap='YlOrRd')
cs1  = ax3.contour(MU,BET,rstar,levels=25,cmap='magma')
cbar = fig3.colorbar(cs0, pad=-0.01)
cbar.ax.set_ylabel('$r_*$',rotation=0,fontsize=30)
ax3.plot(0.1,1,'p',ms=20,mfc='w')
ax3.plot(0.1,1,'p',ms=12,mfc='C9',mec='C9')
ax3.set_xlabel(r'$\mu$')
ax3.set_ylabel(r'$\beta$')
show()
#close()